In [1]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed

from module import models_lib, utils_lib, preprocess_lib, inference_lib

import torch
from pathlib import Path
import subprocess
from module import models_lib

import os
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import subprocess

from openvino.runtime import Core
from module import models_lib


/opt/conda/lib/python3.10/importlib/util.py:247: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  self.__spec__.loader.exec_module(self)


In [2]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 2
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            self.model_path =  "../models/baseline_fold0_0404/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 16

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5
        self.TARGET_DURATION = 5
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 128
        self.FMIN = 50
        self.FMAX = 14000
        
        self.seed = 42


        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [3]:
"TODO: configを2つにわけるべきかも．柔軟に変える方はnotebook側で，固定したい方はmodule側とか"
"TODO: Debugモードになっていたらsubmissionでエラーになる"
"TODO: 疑似ラベル作成のためなら，事前にデータセットを準備するべき"
cfg = CFG(mode='inference', kaggle_notebook=False)

# Set seed
utils_lib.set_seed(cfg.seed)

In [5]:

import json
from datetime import datetime, timezone, timedelta
import os

# === 初期設定 ===
cfg = CFG(mode="inference", kaggle_notebook=False)
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)

# 保存先: "model_path_vino/"
# モデル元ディレクトリ（たとえば ../models/baseline_fold0_0404）
model_dir = Path(cfg.model_path).resolve()  # 絶対パスに変換
vino_dir = model_dir.parent / (model_dir.name + "_vino")  # ← 兄弟フォルダとして _vino を作成

vino_dir.mkdir(parents=True, exist_ok=True)
print(f"📁 Saving ONNX & IR files to: {vino_dir}")

# === dataset-metadata.jsonの作成 ===
japan_time = datetime.now(timezone(timedelta(hours=9)))
current_time = japan_time.strftime('%Y%m%d_%H%M')

# dataset-metadata.jsonを保存
dataset_metadata = {
    "title": f"bc25-models-{current_time}",
    "id": f"ihiratch/bc25-models-{current_time}",
    "licenses": [
        {
            "name": "CC0-1.0"
        }
    ]
}
metadata_path = os.path.join(vino_dir, "dataset-metadata.json")
with open(metadata_path, "w") as f:
    json.dump(dataset_metadata, f, indent=2)
    

# === モデル読み込み（全fold）===
print("📦 Loading all fold models...")
models = models_lib.load_models(cfg, num_classes)

for fold, model in enumerate(models):
    print(f"\n🔁 [Fold {fold}] Converting model...")
    model = model.to("cpu")  
    model.eval()

    # Step 1: Export to ONNX
    onnx_path = vino_dir / f"model_fold{fold}.onnx"
    dummy_input = torch.randn(1, cfg.in_channels, *cfg.TARGET_SHAPE)  # ← ここだけ修正！

    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
        opset_version=11
    )
    print(f"✅ Exported ONNX: {onnx_path.name}")

    # Step 2: Convert to OpenVINO IR
    result = subprocess.run(
        ["ovc", str(onnx_path)],
        cwd=str(vino_dir),  # ← これが重要！
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
        print(f"❌ OpenVINO conversion failed for fold{fold}:")
        print("----- stderr -----")
        print(result.stderr)
        print("----- stdout -----")
        print(result.stdout)
    else:
        print(f"✅ Converted to OpenVINO IR:")
        print(f"   - {(vino_dir / f'model_fold{fold}.xml').resolve()}")
        print(f"   - {(vino_dir / f'model_fold{fold}.bin').resolve()}")
        
        

📁 Saving ONNX & IR files to: /root/program/birdclef-2025/models/baseline_fold0_0404_vino
📦 Loading all fold models...
Found a total of 1 model files.
Loading model: ../models/baseline_fold0_0404/model_fold0.pth


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:215: UserWarning: 
NVIDIA H100 PCIe with CUDA capability sm_90 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA H100 PCIe GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(



🔁 [Fold 0] Converting model...
✅ Exported ONNX: model_fold0.onnx
✅ Converted to OpenVINO IR:
   - /root/program/birdclef-2025/models/baseline_fold0_0404_vino/model_fold0.xml
   - /root/program/birdclef-2025/models/baseline_fold0_0404_vino/model_fold0.bin
